In [ ]:
import io
import zipfile
import requests
import pandas as pd
from pathlib import Path

# 1) parametres globaux
SYMBOLS = ["BTCUSDT", "ETHUSDT", "BNBUSDT", "SOLUSDT", "XRPUSDT", "PENGUUSDT"]
INTERVAL = "1d"
YEARS = [2021, 2022, 2023, 2024, 2025]

# 2) dossier sortie
OUT_DIR = Path("binance_public_data")
OUT_DIR.mkdir(exist_ok=True)

# 3) url binance vision
BASE = "https://data.binance.vision/data/spot/monthly/klines"

# 4) colonnes officielle binance
BINANCE_COLS = [
    "open_time", "open", "high", "low", "close", "volume",
    "close_time", "quote_asset_volume", "number_of_trades",
    "taker_buy_base_volume", "taker_buy_quote_volume", "ignore"
]


def to_datetime_open_time(x):
    # 5) timestamp -> date
    # ms vs us
    x = int(x)
    if x > 10_000_000_000_000:
        return pd.to_datetime(x, unit="us", utc=True).tz_convert(None)
    return pd.to_datetime(x, unit="ms", utc=True).tz_convert(None)


def download_month(symbol, year, month):
    # 6) telechargement mensuel
    m = f"{month:02d}"
    url = f"{BASE}/{symbol}/{INTERVAL}/{symbol}-{INTERVAL}-{year}-{m}.zip"

    r = requests.get(url, timeout=30)
    if r.status_code != 200:
        return None

    # 6b) unzip
    z = zipfile.ZipFile(io.BytesIO(r.content))
    csv_name = [n for n in z.namelist() if n.endswith(".csv")][0]
    raw_csv = z.read(csv_name)

    # 6c) lecture csv
    df = pd.read_csv(io.BytesIO(raw_csv), header=None)
    df.columns = BINANCE_COLS

    # 6d) conversion float
    for c in ["open", "high", "low", "close", "volume"]:
        df[c] = pd.to_numeric(df[c], errors="coerce")

    # 6e) index temps
    df["timestamp"] = df["open_time"].apply(to_datetime_open_time)
    df = df.set_index("timestamp").sort_index()

    # 6f) colonnes utiles
    return df[["open", "high", "low", "close", "volume"]]


def download_symbol(symbol):
    # 7) boucle annees / mois
    parts = []

    for y in YEARS:
        for m in range(1, 13):
            df_m = download_month(symbol, y, m)
            if df_m is not None and not df_m.empty:
                parts.append(df_m)

    if not parts:
        return None

    # 7b) concat
    df = pd.concat(parts).sort_index()

    # 7c) sup doublons
    df = df[~df.index.duplicated(keep="first")]

    # 7d) periode fixe
    df = df.loc["2021-01-01":"2025-12-31"]

    return df


# 8) download univers complet
all_data = {}

for sym in SYMBOLS:
    print("download", sym)

    df = download_symbol(sym)
    if df is None or df.empty:
        print("skip", sym)
        continue

    print("dates", df.index.min(), "->", df.index.max(), "n", len(df))

    # 8b) sauvegarde csv
    out_path = OUT_DIR / f"{sym}_{INTERVAL}_2021_2025.csv"
    df.to_csv(out_path)

    all_data[sym] = df
    print("saved", out_path.as_posix())


download BTCUSDT
dates 2021-01-01 00:00:00 -> 2025-12-31 00:00:00 n 1826
saved binance_public_data/BTCUSDT_1d_2021_2025.csv
download ETHUSDT
dates 2021-01-01 00:00:00 -> 2025-12-31 00:00:00 n 1826
saved binance_public_data/ETHUSDT_1d_2021_2025.csv
download BNBUSDT
dates 2021-01-01 00:00:00 -> 2025-12-31 00:00:00 n 1826
saved binance_public_data/BNBUSDT_1d_2021_2025.csv
download SOLUSDT
dates 2021-01-01 00:00:00 -> 2025-12-31 00:00:00 n 1826
saved binance_public_data/SOLUSDT_1d_2021_2025.csv
download XRPUSDT
dates 2021-01-01 00:00:00 -> 2025-12-31 00:00:00 n 1826
saved binance_public_data/XRPUSDT_1d_2021_2025.csv
download PENGUUSDT
dates 2024-12-17 00:00:00 -> 2025-12-31 00:00:00 n 380
saved binance_public_data/PENGUUSDT_1d_2021_2025.csv


Pour chaque crypto, on calcul d’abord les log-returns à partir des prix de clôture, puis on construit plusieurs mesures de volatilité “simples” (rolling, EWMA, Parkinson, Garman-Klass) pour avoir des benchmarks rapides.

Ensuite, on estime des volatilités “modèles” en recalibrant plusieurs variantes de GARCH (GARCH, GJR, EGARCH) avec deux distributions (normale et Student-t). À chaque date, le modèle est ré-entrainé sur tout l’historique disponible jusqu’à cette date (à partir d’une fenêtre minimale), puis il sort une prévision de volatilité à 1 jour.

Enfin, on compare toutes les volatilités prédites aux volatilités réalisées futures (sur 5 jours et 20 jours) avec des métriques simples (MSE et corrélation), puis il sauvegarde les résultats dans un CSV pour faire un classement des modèles.